## Dash

https://dash.plotly.com/introduction

- hello world (jupyter vs nejupyter)
- callback
- dash + plotly

In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc # dash core components
from dash import html
from dash.dependencies import Input, Output

app = JupyterDash(__name__)

pozdrav = {
    "de" : "Hallo",
    "en" : "Oi, oi",
    "cz" : "Nazdar",
    "jp" : "こんにちは"
}

app.layout = html.Div([
    html.H1("Hallo", id = "header"),
    dcc.RadioItems(
        id = "jazyk",
        options = [
            {"value" : "de", "label" : "Deutsch"},
            {"value" : "en", "label" : "English"},
            {"value" : "cz", "label" : "Česky"},
            {"value" : "jp", "label" : "日本語"}
        ],
        value = "jp"),
    html.Hr()
])

@app.callback(
    Output("header", "children"),
    Input("jazyk", "value"))
def update_hello(val):
    return pozdrav[val]

app.run_server(mode="inline")

In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc # dash core components
from dash import html
from dash.dependencies import Input, Output

import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv("./data/nakazeni-vyleceni-umrti-testy.csv")

df.set_index(['datum'], drop = True, inplace = True)
df.index = pd.to_datetime(df.index)
new_names = ["inf", "cur", "dead", "tests", "ag", "d_inf", "d_cur", "d_dead", "d_tests", "d_ag"]

df.columns = new_names

df = df[["inf", "dead", "d_inf", "d_dead"]]
pretty_names = {
    "inf" : "Nakažení celkem",
    "dead": "Mrtví celkem",
    "d_inf" : "Přírůstek nakažených",
    "d_dead" : "Přírůstek mrtvých"
}

aggr = {"inf" : "sum", "dead" : "sum", "d_inf" : "mean", "d_dead" : "mean"}

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Covid"),
    html.Label([
        "theme",
        dcc.Dropdown(
            id='theme-dropdown', clearable=False,
            value='plotly_white',
            options=[{'label': c, 'value': c} for c in ["plotly_white", "plotly_dark"]],
            )]),
    html.Label([
        "Data",
        dcc.Checklist(
            id='data-checklist',
            value=['d_dead'],
            options=[{'label': pretty_names[d], 'value': d} for d in ["d_inf", "d_dead"]]
            )
    ]),
    html.Label([
        "Klouzavý průměr ",
        dcc.Input(
            id="days", type="number", value = 1,
            min=1, max=10, step=1,
        )
    ]),
    dcc.Graph(id="graph"),
    html.Hr(),
    html.Label([
        "Data",
        dcc.Checklist(
            id='data-cummulative',
            value=['dead'],
            options=[{'label': pretty_names[d], 'value': d} for d in ["inf", "dead"]],
            )
    ]),
    dcc.Checklist(
        id='logscale',
        value = [],
        options=[{'label': "Logaritmická škála", 'value': "logscale_y"}],
    ),
    dcc.Graph(id="graph2"),
])

@app.callback(
    Output("graph", "figure"),
    Input("theme-dropdown", "value"),
    Input("data-checklist", "value"),
    Input("days", "value"))
def update_figure1(theme, data, days):
    if days is None:
        days = 1
    fig = go.Figure()
    for dt in data:
        fig.add_trace(go.Scatter(x = df.index, y = df[dt], name = pretty_names[dt]))

    if days > 1:
        resampled = df.resample("%dD" % days).agg(aggr)
        for dt in data:
            fig.add_trace(go.Scatter(x = resampled.index, y = resampled[dt],
                                     name = pretty_names[dt] + f" {days}D avg"))
        
    fig.update_layout(template = theme, title = "COVID19 - Přírůstky")
    return fig


@app.callback(
    Output("graph2", "figure"),
    Input("theme-dropdown", "value"),
    Input("data-cummulative", "value"),
    Input("logscale", "value"))
def update_figure2(theme, data, logscale):
    if len(logscale) == 0:
        logscale = False
    fig = go.Figure()

    for dt in data:
        fig.add_trace(go.Scatter(x = df.index, y = df[dt], name = pretty_names[dt]))
    
    if logscale:
        fig.update_yaxes(type="log")    
    fig.update_layout(template = theme, title = "COVID19 - Kumulativní počty")
    return fig

app.run_server(mode="inline")

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html

from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)


app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size", size = "size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline')

## Dash Bootstrap

https://getbootstrap.com/

Implementováno v Dash

http://dash-bootstrap-components.opensource.faculty.ai/

In [6]:
import dash
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc
from jupyter_dash import JupyterDash
import pandas as pd
from dash.dependencies import Input, Output
import plotly.graph_objects as go

df = pd.read_csv("data/nakazeni-vyleceni-umrti-testy.csv")

df.set_index(['datum'], drop = True, inplace = True)
df.index = pd.to_datetime(df.index)
new_names = ["inf", "cur", "dead", "tests", "ag", "d_inf", "d_cur", "d_dead", "d_tests", "d_ag"]

df.columns = new_names

df = df[["inf", "dead", "d_inf", "d_dead"]]
pretty_names = {
    "inf" : "Nakažení celkem",
    "dead": "Mrtví celkem",
    "d_inf" : "Přírůstek nakažených",
    "d_dead" : "Přírůstek mrtvých"
}

aggr = {"inf" : "sum", "dead" : "sum", "d_inf" : "mean", "d_dead" : "mean"}

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

carousel = dbc.Carousel(
    items=[
        {"key": "1", "src": "assets/covid_01.jpeg", "img_style": {"object-fit": "contain", "max-height": "250px"}},
        {"key": "2", "src": "assets/covid_02.jpeg", "img_style": {"object-fit": "contain", "max-height": "250px"}},
        {"key": "3", "src": "assets/covid_03.jpeg", "img_style": {"object-fit": "contain", "max-height": "250px"}},
    ],
    controls=False,
    indicators=False,
    interval=2000,
    ride="carousel",
    
)

controls = html.Div([
    dbc.InputGroup([
        dbc.InputGroupText("Klouzavý průměr"),
        dbc.Input(
            id="days",
            type="number",
            value=1,
            min=1,
            max=10,
            step=1
        )
    ]),
    dbc.InputGroup([
        dbc.InputGroupText("Data set"),
        dbc.Checklist(
        options=[{'label': pretty_names[d], 'value': d} for d in ["d_inf", "d_dead"]],
        value=["d_dead"],
        id="data-checklist",
        ),
    ]),
    dbc.InputGroup([
        dbc.InputGroupText("Plotly theme"),
        dbc.Select(
            id='theme-dropdown',
            value='plotly_white',
            options=[{'label': c, 'value': c} for c in ["plotly_white", "plotly_dark"]],

        )
    ])
])

controls2 = html.Div([
     dbc.InputGroup([
        dbc.InputGroupText("Data"),
        dbc.Checklist(
            id='data-cummulative',
            value=['dead'],
            options=[{'label': pretty_names[d], 'value': d} for d in ["inf", "dead"]],
        )
    ]),
    dbc.Checklist(
        id='logscale',
        value = [],
        options=[{'label': "Logaritmická škála", 'value': "logscale_y"}],
    ),
])
accordion = html.Div(
    dbc.Accordion(
        [
            dbc.AccordionItem(
                [
                    controls,
                    dcc.Graph(id="graph")
                ],
                title="Covid",
            ),
            dbc.AccordionItem(
                [
                    controls2,
                    dcc.Graph(id="graph2")
                ],
                title="Taky covid",
            )
        ],
    )
)

app.layout = dbc.Container([carousel, accordion])

@app.callback(
    Output("graph", "figure"),
    Input("theme-dropdown", "value"),
    Input("data-checklist", "value"),
    Input("days", "value"))
def update_figure1(theme, data, days):
    if days is None:
        days = 1
    fig = go.Figure()
    for dt in data:
        fig.add_trace(go.Scatter(x = df.index, y = df[dt], name = pretty_names[dt]))

    if days > 1:
        resampled = df.resample("%dD" % days).agg(aggr)
        for dt in data:
            fig.add_trace(go.Scatter(x = resampled.index, y = resampled[dt],
                                     name = pretty_names[dt] + f" {days}D avg"))
        
    fig.update_layout(template = theme, title = "COVID19 - Přírůstky")
    return fig

@app.callback(
    Output("graph2", "figure"),
    Input("theme-dropdown", "value"),
    Input("data-cummulative", "value"),
    Input("logscale", "value"))
def update_figure2(theme, data, logscale):
    if len(logscale) == 0:
        logscale = False
    fig = go.Figure()

    for dt in data:
        fig.add_trace(go.Scatter(x = df.index, y = df[dt], name = pretty_names[dt]))
    
    if logscale:
        fig.update_yaxes(type="log")    
    fig.update_layout(template = theme, title = "COVID19 - Kumulativní počty")
    return fig


app.run_server()

/data/unicorn/pythonbook/venv/lib/python3.10/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



TypeError: 'NoneType' object cannot be interpreted as an integer